In [1]:
import joblib
import pandas as pd
import nltk
import re
import itertools
import random
import numpy as np

In [2]:
data = pd.read_csv('ASAP7 Train Set.tsv', sep='\t')
data_100 = data[:100]
data_100.head()

,essay_id,essay,domain1_score
0,19441,Patience is tough to achieve sometimes. But th...,22
1,18068,Patience a helpful trait to get you through li...,24
2,19000,There were many of times when I was patient. T...,18
3,18339,A time I was patiet was when I was wating for ...,24
4,18297,Do you ever fell like your @CAPS1 is going to ...,21


In [3]:
corrections = joblib.load('files/json_responses_train_100')
#correction_train_100essay_5corr = joblib.load('correction_train_100essay_5corr')

In [4]:
# Function to sample 5 dictionaries from each inner list
def sample_five_from_each(inner_lists):
    return [random.sample(inner_list, min(len(inner_list), 5)) for inner_list in inner_lists]

# Sample 5 dictionaries from each inner list
sampled_corrections = sample_five_from_each(corrections)

print(np.array(sampled_corrections).shape)

# Print the results
for idx, sampled_list in enumerate(sampled_corrections):
    print(f"Sampled data for list {idx + 1}:")
    for item in sampled_list:
        print(item)
    print()


(100, 5)
Sampled data for list 1:
{'original': 'succed', 'correction': 'succeed', 'type': 'Spelling', 'sentence': 2}
{'original': 'keep trying and your', 'correction': 'keep trying and you', 'type': 'Grammatical', 'sentence': 2}
{'original': 'any ways', 'correction': 'anyway', 'type': 'Word Choice', 'sentence': 21}
{'original': 'was had to pride', 'correction': 'was proud', 'type': 'Grammatical', 'sentence': 11}
{'original': "wat don't", 'correction': "wasn't", 'type': 'Grammatical', 'sentence': 11}

Sampled data for list 2:
{'original': 'A patient;', 'correction': 'A patient:', 'type': 'Grammatical', 'sentence': 3}
{'original': 'a helpful', 'correction': 'is a helpful', 'type': 'Grammatical', 'sentence': 1}
{'original': 'even far away', 'correction': 'farther away', 'type': 'Word Choice', 'sentence': 17}
{'original': 'types of patient', 'correction': 'types of patients', 'type': 'Grammatical', 'sentence': 4}
{'original': 'tetns vaccined', 'correction': 'tetanus vaccine', 'type': 'Spel

In [5]:
joblib.dump(sampled_corrections, 'files/correction_train_100essay_5corr')

['files/correction_train_100essay_5corr']

In [6]:
# Function to apply corrections to the essay
def apply_corrections_grammar(essay, corrections):
    # Tokenize the essay into sentences using NLTK
    sentences = nltk.sent_tokenize(essay)

    # Create a list to store corrected sentences
    corrected_sentences = []

    # Iterate through sentences
    for i, sentence in enumerate(sentences):
        # Apply corrections to the current sentence
        for correction in corrections:
            # Adjust the index by subtracting 1 to match the enumeration starting from 0
            if correction['sentence'] == i + 1:
                # Replace all occurrences of the incorrect text with the correction
                #sentence = sentence.replace(correction['original'], correction['correction'])
                 # Use regular expression with word boundaries to replace only whole words
                sentence = re.sub(r'\b' + re.escape(correction['original']) + r'\b', correction['correction'], sentence)
        # Add the corrected sentence to the list
        corrected_sentences.append(sentence)

    # Join the corrected sentences to form the final essay
    corrected_essay = ' '.join(corrected_sentences)
    return corrected_essay

# Function to generate all combinations of corrections and apply them
def generate_corrected_versions(essay, corrections):
    corrected_versions = []
    max_corrections = len(corrections)

    # Iterate over different numbers of corrections to apply
    for num_corrections in range(1, max_corrections + 1):
        for combination in itertools.combinations(corrections, num_corrections):
            corrected_essay = apply_corrections_grammar(essay, combination)
            corrected_versions.append(corrected_essay)
  
    return corrected_versions

# # Example usage
# essay = "This is a tess essay. The break through was amazing. The possitive effects were clear."
# corrections = [
#     {'original': 'tess', 'correction': 'test', 'type': 'Spelling', 'sentence': 1},
#     {'original': 'break through', 'correction': 'breakthrough', 'type': 'Spelling', 'sentence': 2},
#     {'original': 'possitive', 'correction': 'positive', 'type': 'Spelling', 'sentence': 3},
# ]

# # Generate corrected versions with combinations of 1, 2, and 3 corrections
# corrected_versions = generate_corrected_versions(essay, corrections)
# corrected_versions

In [7]:
correction_train_100essay_5corr = joblib.load('files/correction_train_100essay_5corr')

corrected_train_100data_5corr = []

for index, row in data_100.iterrows():
    try:
        correted_essays = generate_corrected_versions(row['essay'], correction_train_100essay_5corr[index]) 
        # Append the result to the list
        corrected_train_100data_5corr.append(correted_essays)
    except Exception as e:
        print(f"Error processing row {index}: {e}")
        corrected_train_100data_5corr.append(None)  # Append None or an empty dict if an error occurs

In [8]:
np.array(corrected_train_100data_5corr).shape

(100, 31)

In [9]:
joblib.dump(corrected_train_100data_5corr, 'files/corrected_train_100data_5corr')

['files/corrected_train_100data_5corr']